In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from keras.src.metrics.accuracy_metrics import accuracy

torch.cuda.is_available()


False

In [2]:
# Combined dataset: bikes for short distances, cars for longer ones
distances = torch.tensor([
    [1.0], [1.5], [2.0], [2.5], [3.0], [3.5], [4.0], [4.5], [5.0], [5.5],
    [6.0], [6.5], [7.0], [7.5], [8.0], [8.5], [9.0], [9.5], [10.0], [10.5],
    [11.0], [11.5], [12.0], [12.5], [13.0], [13.5], [14.0], [14.5], [15.0], [15.5],
    [16.0], [16.5], [17.0], [17.5], [18.0], [18.5], [19.0], [19.5], [20.0]
], dtype=torch.float32)

# Corresponding delivery times in minutes
times = torch.tensor([
    [6.96], [9.67], [12.11], [14.56], [16.77], [21.7], [26.52], [32.47], [37.15], [42.35],
    [46.1], [52.98], [57.76], [61.29], [66.15], [67.63], [69.45], [71.57], [72.8], [73.88],
    [76.34], [76.38], [78.34], [80.07], [81.86], [84.45], [83.98], [86.55], [88.33], [86.83],
    [89.24], [88.11], [88.16], [91.77], [92.27], [92.13], [90.73], [90.39], [92.98]
], dtype=torch.float32)

In [3]:
distances_std = distances.std()
distances_mean = distances.mean()

times_std = times.std()
times_mean = times.mean()

distances_norm = (distances-distances_mean)/distances_std
times_norm = (times - times_mean)/times_std

In [4]:
model = nn.Sequential(
    nn.Linear(1,3),
    nn.ReLU(),
    nn.Linear(3,1)
)

In [5]:
loss_function = nn.MSELoss()
gradient = optim.SGD(model.parameters(),lr=0.01)

In [6]:
for epoches in range(1000):
    gradient.zero_grad()

    output = model(distances_norm)

    loss = loss_function(output, times_norm)

    loss.backward()

    gradient.step()

    if (epoches + 1) % 50 == 0:
        print(f"Epoch : {epoches+1} and Loss :{loss.item()}" )
        

print("\nTraining Complete.")
print(f"\nFinal Loss: {loss.item()}")


Epoch : 50 and Loss :0.5651528835296631
Epoch : 100 and Loss :0.2961590588092804
Epoch : 150 and Loss :0.14635375142097473
Epoch : 200 and Loss :0.07368024438619614
Epoch : 250 and Loss :0.04094141349196434
Epoch : 300 and Loss :0.026829881593585014
Epoch : 350 and Loss :0.020816726610064507
Epoch : 400 and Loss :0.017968837171792984
Epoch : 450 and Loss :0.016489338129758835
Epoch : 500 and Loss :0.015624321065843105
Epoch : 550 and Loss :0.015039408579468727
Epoch : 600 and Loss :0.01459487620741129
Epoch : 650 and Loss :0.01423056423664093
Epoch : 700 and Loss :0.013918861746788025
Epoch : 750 and Loss :0.013520519249141216
Epoch : 800 and Loss :0.013058059848845005
Epoch : 850 and Loss :0.01264489907771349
Epoch : 900 and Loss :0.01227481383830309
Epoch : 950 and Loss :0.01194242388010025
Epoch : 1000 and Loss :0.011643080040812492

Training Complete.

Final Loss: 0.011643080040812492


In [7]:
new_d = 5.1
""

In [8]:
with torch.no_grad():
    new_distance = torch.tensor([[new_d]], dtype=torch.float32)

    new_dist_norm = (new_distance - distances_mean)/distances_std

    output = model(new_dist_norm)

    outputnorm = (output*times_std) + times_mean

    print(f"You need {outputnorm} to reach {new_d}")

You need tensor([[37.4755]]) to reach 5.1


In [9]:
path = "./datasets./data_with_features.csv"
data = pd.read_csv(path)
data.shape

(100, 4)

In [10]:
data.head()

,distance_miles,time_of_day_hours,is_weekend,delivery_time_minutes
0,1.60,8.20,0,7.22
1,13.09,16.80,1,32.41
2,6.97,8.02,1,17.47
3,10.66,16.07,0,37.17
4,18.24,13.47,0,38.36


In [11]:
def rush_hour_features(hours, weekends):
    is_morning_rush = (hours >= 8) & (hours <= 10)
    is_evening_rush = (hours >= 16) & (hours <= 19)

    is_weekday = weekends == 0

    rush_hour_mask = (is_morning_rush | is_evening_rush) & is_weekday

    return rush_hour_mask.unsqueeze(1).float()


In [12]:
def prep_data(df):

    all_values = df.values

    tensors = torch.tensor(all_values, dtype = torch.float32)

    raw_dist = tensors[:, 0]
    raw_hours = tensors[:, 1]
    raw_weekends=tensors[:, 2]
    raw_targets = tensors[:, 3]

    is_rush_hr = rush_hour_features(raw_hours, raw_weekends)  # Correct
    dist = raw_dist.unsqueeze(1)
    hrs = raw_hours.unsqueeze(1)
    weeknd = raw_weekends.unsqueeze(1)
    rush_hr = is_rush_hr

    dist_std, dist_mean = dist.std(), dist.mean()
    hrs_std, hrs_mean = hrs.std(), hrs.mean()

    dist_norm = (dist - dist_mean)/ dist_std
    hrs_norm = (hrs- hrs_mean)/ hrs_std

    features = torch.cat([
        dist_norm,
        hrs_norm,
        weeknd,
        rush_hr],
        dim = 1
    )

    targets = raw_targets.unsqueeze(1)

    return_dict = {
        'full_tensor': tensors,
        'raw_distances': raw_dist,
        'raw_hours': raw_hours,
        'raw_weekends': raw_weekends,
        'raw_targets': raw_targets,
        'distances_col': dist,
        'hours_col': hrs,
        'weekends_col': weeknd,
        'rush_hour_col': rush_hr
    }

    return features, targets, return_dict

In [13]:
# Create a small test DataFrame with the first 5 entries
test_df = data.head(5).copy()

# Print the "Before" state as a raw tensor
raw_test_tensor = torch.tensor(data.values, dtype=torch.float32)
print("--- Raw Tensor (Before Preparation) ---\n")
print(f"Shape: {raw_test_tensor.shape}")
print("Values:\n", raw_test_tensor)
print("\n" + "="*50 + "\n")

# Run the function to get the prepared "after" tensors
test_features, test_targets, _ = prep_data(test_df)

# Print the "After" state
print("--- Prepared Tensors (After Preparation) ---")
print("\n--- Prepared Features ---\n")
print(f"Shape: {test_features.shape}")
print("Values:\n", test_features)

print("\n--- Prepared Targets ---")
print(f"Shape: {test_targets.shape}")
print("Values:\n", test_targets)

--- Raw Tensor (Before Preparation) ---

Shape: torch.Size([100, 4])
Values:
 tensor([[ 1.6000,  8.2000,  0.0000,  7.2200],
        [13.0900, 16.8000,  1.0000, 32.4100],
        [ 6.9700,  8.0200,  1.0000, 17.4700],
        [10.6600, 16.0700,  0.0000, 37.1700],
        [18.2400, 13.4700,  0.0000, 38.3600],
        [ 5.7400, 16.5900,  0.0000, 29.0600],
        [ 8.8000, 12.2500,  0.0000, 23.9400],
        [15.3600, 11.7600,  1.0000, 32.4000],
        [ 5.3500,  9.4200,  0.0000, 17.0600],
        [ 2.4600, 14.4400,  0.0000, 14.0900],
        [ 6.5100,  8.0000,  0.0000, 33.3800],
        [ 4.0600,  9.3300,  1.0000, 17.3800],
        [18.6600, 14.8600,  1.0000, 36.7500],
        [16.3500, 19.0900,  0.0000, 38.8600],
        [13.0300, 13.4200,  0.0000, 32.5500],
        [17.5600, 18.9200,  0.0000, 61.8700],
        [16.2700, 15.2600,  0.0000, 38.0800],
        [ 4.5400,  9.1000,  0.0000, 24.1200],
        [17.9600, 15.2000,  0.0000, 43.2100],
        [11.2500,  8.0000,  0.0000, 41.0300],
  

In [14]:
features, targets, _ = prep_data(data)

In [15]:
def new_model():

    model = nn.Sequential(
        nn.Linear(4,64),
        nn.ReLU(),
        nn.Linear(64,32),
        nn.ReLU(),
        nn.Linear(32,1)
    )
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    loss_function = nn.MSELoss()


    return model, optimizer, loss_function

In [16]:
model, optimizer, loss_function = new_model()

print(f"{'='*30}\nInitialized Model Architecture\n{'='*30}\n{model}")
print(f"\n{'='*30}\nOptimizer\n{'='*30}\n{optimizer}")
print(f"\n{'='*30}\nLoss Function\n{'='*30}\n{loss_function}")

Initialized Model Architecture
Sequential(
  (0): Linear(in_features=4, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=1, bias=True)
)

Optimizer
SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

Loss Function
MSELoss()


In [17]:
def tarin(features, targets, epochs, verbose = True):

    losses= []

    model, optimizer, loss_function = new_model()

    for epoch in range(epochs):
        output = model(features)
        loss = loss_function(output, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

         # Every 5000 epochs, record the loss and print the progress
        if (epoch + 1) % 5000 == 0:
            losses.append(loss.item())
            if verbose:
                print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")
    
    return model, losses

In [18]:
test_model = tarin(features, targets,10000)

Epoch [5000/10000], Loss: 4.0634
Epoch [10000/10000], Loss: 1.9242


In [19]:
# Training loop
model, loss = tarin(features, targets, 30000)

Epoch [5000/30000], Loss: 4.2364
Epoch [10000/30000], Loss: 2.9529


KeyboardInterrupt: 

In [20]:
# Disable gradient calculation for efficient predictions
with torch.no_grad():
    # Perform a forward pass to get model predictions
    predicted_outputs = model(features)

*DAY 2*


In [1]:
import torch
import torch.nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using device: CUDA")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using device: MPS (Apple Silicon GPU)")
else:
    device = torch.device("cpu")
    print(f"Using device: CPU")

Using device: CPU


In [3]:
path = "./datasets"

In [4]:
train_dataset_no_trsfm = torchvision.datasets.MNIST(
    root = path,
    train = True,
    download = True
)

In [5]:
image_pil, label = train_dataset_no_trsfm[0]

print(f"Image type:    {type(image_pil)}")
print(f"Image Dimension: {image_pil.size}")
print(f"Label type:  {type(label)}")
print(f"Label Vlaue:  {label}")

Image type:    <class 'PIL.Image.Image'>
Image Dimension: (28, 28)
Label type:  <class 'int'>
Label Vlaue:  5


In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.01307), (0.3081))
])

In [7]:
traind_data= torchvision.datasets.MNIST(
    root = path,
    train = True,
    download  = True,
    transform = transform
)

In [8]:
# Access the first item again
image_tensor, label = traind_data[0]

print(f"Image Type:                   {type(image_tensor)}")
# Since the `image` is now a PyTorch Tensor, its dimensions are accessed using the .shape attribute.
print(f"Image Shape After Transform:  {image_tensor.shape}")
print(f"Label Type:                   {type(label)}")
print(f"Label value:                  {label}")

Image Type:                   <class 'torch.Tensor'>
Image Shape After Transform:  torch.Size([1, 28, 28])
Label Type:                   <class 'int'>
Label value:                  5


In [9]:
test_data = torchvision.datasets.MNIST(
    root = path,
    train = False,
    download = True,
    transform = transform
)

In [10]:
train_dt_ldr = DataLoader(traind_data, batch_size = 32, shuffle = True)

test_dt_ldr = DataLoader(test_data, batch_size = 64, shuffle = False)

In [17]:
import torch.nn as nn
class SimpleMNISTNN(nn.Module):
    def __init__(self):
        super(SimpleMNISTNN, self).__init__()

        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(784,128),
            nn.ReLU(),
            nn.Linear(128,10)
        )

    def forward(self, x):

        x = self.flatten(x)
        x = self.layers(x)
        return x

In [18]:
model = SimpleMNISTNN()

loss_function = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [26]:
def train_epoches(model, loss_function, optimizer, train_loader, device):
    model.to(device)
    model.train()

    epoch_loss = 0.0
    running_loss = 0.0
    total_preds = 0
    num_correct_preds =0
    total_batches = len(train_loader)

    for batch_idx, (input, targets) in enumerate(train_loader):

        input, targets = input.to(device), targets.to(device)

        optimizer.zero_grad()

        output = model(input)
        loss = loss_function(output, targets)
        loss.backward()
        optimizer.step()

        loss_valu = loss.item()
        epoch_loss +=loss_valu
        running_loss += loss_valu

        _, predicted_indices = output.max(1)
        batch_size = targets.size(0)
        total_preds += batch_size
        num_correct_preds += predicted_indices.eq(targets).sum().item()

        if (batch_idx + 1) % 134 == 0 or (batch_idx + 1) == total_batches:
            # Calculate average loss and accuracy for the current interval
            avg_running_loss = running_loss / 134
            accuracy = 100 * (num_correct_preds / total_preds)

            # Print the progress update
            print(f'\tStep {batch_idx + 1}/{total_batches} - Loss: {avg_running_loss:.3f} | Acc: {accuracy:.2f}%')

            # Reset the trackers for the next reporting interval
            running_loss = 0.0
            num_correct_preds = 0
            total_preds = 0

    # Calculate the average loss for the entire epoch
    avg_epoch_loss = epoch_loss / total_batches
    # Return the trained model and the average epoch loss
    return model, avg_epoch_loss


In [30]:
def evaluate(model, test_loader, device):
    model.eval()
    model.to(device)

    num_crct_preds = 0
    total_predfs = 0

    with torch.no_grad():

        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            output = model(inputs)
            _, predicted_indices = torch.max(output, 1)
            batch_size = targets.size(0)
            total_predfs += batch_size
            num_crct_preds += predicted_indices.eq(targets).sum().item()

        accuracy_percent = (num_crct_preds/total_predfs)*100
        print((f'\tAccuracy - {accuracy_percent:.2f}%'))

        return accuracy_percent

In [31]:
num_epochs = 5


train_losss = []
test_acc = []

for epoch in range(num_epochs):

    print(f"Epoch {epoch+1}")

    trained_model, loss = train_epoches(model, loss_function, optimizer, train_dt_ldr, device)

    train_losss.append(loss)
    print(
         f"Testing Epoch {epoch+1}")

    accuracy = evaluate(trained_model, test_dt_ldr, device)
    test_acc.append(accuracy)

Epoch 1
	Step 134/1875 - Loss: 0.062 | Acc: 98.34%
	Step 268/1875 - Loss: 0.059 | Acc: 97.95%
	Step 402/1875 - Loss: 0.068 | Acc: 97.74%
	Step 536/1875 - Loss: 0.070 | Acc: 97.90%
	Step 670/1875 - Loss: 0.064 | Acc: 97.90%
	Step 804/1875 - Loss: 0.068 | Acc: 97.71%
	Step 938/1875 - Loss: 0.066 | Acc: 97.85%
	Step 1072/1875 - Loss: 0.060 | Acc: 98.04%
	Step 1206/1875 - Loss: 0.064 | Acc: 97.99%
	Step 1340/1875 - Loss: 0.062 | Acc: 97.99%
	Step 1474/1875 - Loss: 0.071 | Acc: 97.55%
	Step 1608/1875 - Loss: 0.064 | Acc: 97.85%
	Step 1742/1875 - Loss: 0.067 | Acc: 97.81%
	Step 1875/1875 - Loss: 0.057 | Acc: 98.10%
Testing Epoch 1
	Accuracy - 97.34%
Epoch 2
	Step 134/1875 - Loss: 0.046 | Acc: 98.48%
	Step 268/1875 - Loss: 0.040 | Acc: 98.60%
	Step 402/1875 - Loss: 0.053 | Acc: 98.44%
	Step 536/1875 - Loss: 0.057 | Acc: 98.39%
	Step 670/1875 - Loss: 0.043 | Acc: 98.32%
	Step 804/1875 - Loss: 0.057 | Acc: 98.16%
	Step 938/1875 - Loss: 0.044 | Acc: 98.48%
	Step 1072/1875 - Loss: 0.050 | Acc: 98